A Monte Carlo simulation is a model used to predict the probability of different outcomes when the intervention of random variables is present.

Monte Carlo simulations help to explain the impact of risk and uncertainty in prediction and forecasting models.

The basis of a Monte Carlo simulation involves assigning multiple values to an uncertain variable to achieve multiple results and then averaging the results to obtain an estimate.

We start by importing the libraries.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set_style('whitegrid')

Let's assume that our historical percent to target performance is centered around a a mean of 100% and standard deviation of 10%. We can define those variables as well as the number of sales reps and simulations to model.

In [2]:
avg = 1
std_dev = .1
num_reps = 500
num_simulations = 1000

Using numpy, we can generate a list of percentages that can replicate our historical normal distribution.

In [3]:
pct_to_target = np.random.normal(avg, std_dev, num_reps).round(2)

We can now create variables for sales target and group them into buckets.

In [4]:
sales_target_values = [75_000, 100_000, 200_000, 300_000, 400_000, 500_000]
sales_target_prob = [.3, .3, .2, .1, .05, .05]
sales_target = np.random.choice(sales_target_values, num_reps, p=sales_target_prob)

We now turn the data into a dataframe.

In [5]:
df = pd.DataFrame(index=range(num_reps), data={'Pct_To_Target': pct_to_target,
                                               'Sales_Target': sales_target})

df['Sales'] = df['Pct_To_Target'] * df['Sales_Target']

df.head()

,Pct_To_Target,Sales_Target,Sales
0,1.03,100000,103000.0
1,1.14,75000,85500.0
2,1.06,200000,212000.0
3,1.05,200000,210000.0
4,1.04,100000,104000.0


We define the function to calculate the commission rate as follows:

In [6]:
def calc_commission_rate(x):
    """ Return the commission rate based on the table:
    0-90% = 2%
    91-99% = 3%
    >= 100 = 4%
    """
    if x <= .90:
        return .02
    if x <= .99:
        return .03
    else:
        return .04

Now we can create the commission rate.

In [7]:
df['Commission_Rate'] = df['Pct_To_Target'].apply(calc_commission_rate)
df['Commission_Amount'] = df['Commission_Rate'] * df['Sales']
df.head()

,Pct_To_Target,Sales_Target,Sales,Commission_Rate,Commission_Amount
0,1.03,100000,103000.0,0.04,4120.0
1,1.14,75000,85500.0,0.04,3420.0
2,1.06,200000,212000.0,0.04,8480.0
3,1.05,200000,210000.0,0.04,8400.0
4,1.04,100000,104000.0,0.04,4160.0


Let's loop through the total number of iterations (1000).

In [8]:
# Define a list to keep all the results from each simulation that we want to analyze
all_stats = []

# Loop through many simulations
for i in range(num_simulations):

    # Choose random inputs for the sales targets and percent to target
    sales_target = np.random.choice(sales_target_values, num_reps, p=sales_target_prob)
    pct_to_target = np.random.normal(avg, std_dev, num_reps).round(2)

    # Build the dataframe based on the inputs and number of reps
    df = pd.DataFrame(index=range(num_reps), data={'Pct_To_Target': pct_to_target,
                                                   'Sales_Target': sales_target})

    # Back into the sales number using the percent to target rate
    df['Sales'] = df['Pct_To_Target'] * df['Sales_Target']

    # Determine the commissions rate and calculate it
    df['Commission_Rate'] = df['Pct_To_Target'].apply(calc_commission_rate)
    df['Commission_Amount'] = df['Commission_Rate'] * df['Sales']

    # We want to track sales,commission amounts and sales targets over all the simulations
    all_stats.append([df['Sales'].sum().round(0),
                      df['Commission_Amount'].sum().round(0),
                      df['Sales_Target'].sum().round(0)])

In [9]:
results_df = pd.DataFrame.from_records(all_stats, columns=['Sales',
                                                           'Commission_Amount',
                                                           'Sales_Target'])

In [10]:
results_df.describe().round(2).style.format('{:,}')

,Sales,Commission_Amount,Sales_Target
count,"1,000.0","1,000.0","1,000.0"
mean,"83,693,217.25","2,856,644.39","83,703,725.0"
std,"2,673,028.24","101,803.56","2,624,917.51"
min,"75,082,000.0","2,530,685.0","74,900,000.0"
25%,"81,785,937.5","2,786,676.25","81,868,750.0"
50%,"83,733,375.0","2,858,976.0","83,775,000.0"
75%,"85,439,500.0","2,922,395.5","85,400,000.0"
max,"92,436,250.0","3,173,200.0","92,100,000.0"


In [11]:
#results_df.describe().round(2)

Useful resources:
- https://pbpython.com/monte-carlo.html